# Modern Portfolio Theory

Modern Portfolio Theory
Modern portfolio theory also popularly called Mean-Variance Portfolio Theory (MVP) is a major breakthrough in finance. It is based on the premise that returns are normally distributed and by looking at mean and variance, we can essentially describe the distribution of end-of-period wealth.

The basic idea of this theory is to achieve diversification by constructing a portfolio for a minimal portfolio risk or maximal portfolio returns. Accordingly, the Efficient Frontier is a set of optimal portfolios in the risk-return spectrum, and portfolios located under the Efficient Frontier curve are considered sub-optimal.

This means that the portfolios on the frontier offered

Highest expected return for a given level of risk

Lowest level of risk for a given level of expected returns


In essence, the investors' goal should be to select a level of risk that he/she is comfortable with and then find a portfolio that maximizes returns based on the selected risk level.

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from numpy.linalg import multi_dot
import cufflinks as cf
import plotly.express as px
import datetime

# Set the offline mode for cufflinks
cf.set_config_file(offline=True, dimensions=(1000, 600))
px.defaults.width, px.defaults.height = 1000, 600

# Set the plotly template
px.defaults.template = "plotly_white"

# Set the random seed to ensure reproducibility
np.random.seed(0)

Retrive Data
We will retrieve price data from a list of stocks using the yfinance library. We will use the adjusted closing price for our analysis.


In [2]:
assets = ['AAPL', 'MSFT', 'AMZN', 'GOOG']

# Number of assets
num_assets = len(assets)

# Number of simulations for the optimisation process
num_simulations = 10000

# Start date, end date and trading days in a year
end_date = datetime.datetime(2018, 12, 31)
start_date = end_date - datetime.timedelta(days=365)
TRADING_DAYS = 252

# Download the data
# df = pd.read_csv('asset_close_prices.csv', index_col=0, parse_dates=True)
df = yf.download(assets, start=start_date, end=end_date)['Adj Close']
df.head()

[*********************100%***********************]  4 of 4 completed


,AAPL,AMZN,GOOG,MSFT
Date,,,,
2018-01-02,40.831585,59.450500,53.250000,80.562042
2018-01-03,40.824474,60.209999,54.124001,80.936974
2018-01-04,41.014111,60.479500,54.320000,81.649338
2018-01-05,41.481068,61.457001,55.111500,82.661659
2018-01-08,41.326996,62.343498,55.347000,82.745995


In [3]:
# Plot price history for 
df.iplot(kind='line')

In [4]:
# Dataframe of returns and volatility
returns = df.pct_change().dropna()
annual_returns = round(returns.mean() * TRADING_DAYS * 100, 2)
annual_volatility = round(returns.std() * np.sqrt(TRADING_DAYS) * 100, 2)

df1 = pd.DataFrame({
'Annualised Return': annual_returns,
'Annualised Volatility': annual_volatility
})

df1.head()

,Annualised Return,Annualised Volatility
AAPL,-4.24,28.78
AMZN,28.56,36.18
GOOG,1.29,28.19
MSFT,21.46,28.35


In [5]:
# Plot annualized return and volatility
df1.iplot(kind='bar', shared_xaxes=True, orientation='h', title='Annualized Return and Volatility')

### Portfolio Performance
Let's first assume an equal weights of assets in our portfolio and define some functions to analyse the performance of our portfolio.

In [6]:
## Define functions for portfolio return, volatility and Sharpe ratio, used to calculate portfolio performance and optimise the portfolio
def portfolio_Volatility(weights: np.ndarray):
    return np.sqrt(np.dot(weights.T, np.dot(returns.cov() * TRADING_DAYS, weights))) 

def portfolio_Variance(weights: np.ndarray):
    return portfolio_Volatility(weights)**2

def portfolio_Return(weights: np.ndarray):
    return np.sum(returns.mean() * weights) * TRADING_DAYS

def portfolio_Negative_Max_Sharpe_Ratio(weights: np.ndarray):
    return -(portfolio_Return(weights) / portfolio_Volatility(weights))

# Create a function to calculate portfolio return, volatility and Sharpe ratio
def portfolio_Performance(weights: list):
    weights = np.array(weights)
    portfolio_return = portfolio_Return(weights)
    portfolio_volatility = portfolio_Volatility(weights)
    portfolio_variance = portfolio_volatility**2
    portfolio_sharpe = portfolio_return/portfolio_volatility

    # Return a dataframe of portfolio return, volatility and Sharpe ratio
    return pd.DataFrame({
        'Return': portfolio_return,
        'Volatility': portfolio_volatility,
        'Sharpe Ratio': portfolio_sharpe
    }, index=[0])

### Portfolio Statistics
Consider a portfolio which is fully invested in risky assets. Let $w$ and $\mu$ be the vector of weights and mean returns of $n$ assets.

$$
w=\left(\begin{array}{c}
w_{1} \\
w_{2} \\
\vdots \\
w_{n}
\end{array}\right) ; \mu=\left(\begin{array}{c}
\mu_{1} \\
\mu_{2} \\
\vdots \\
\mu_{n}
\end{array}\right)
$$

where the $\sum_{i=1}^{n} w_{i}=1$

Expected Portfolio Return is then the dot product of the expected returns and their weights.

$$
\mu_{\pi}=w^{T} \cdot \mu
$$

which is also equivalent to the $\sum_{i=1}^{n} w_{i} \mu_{i}$

Expected Portfolio Variance is then the multidot product of weights and the covariance matrix.

$$
\sigma_{\pi}^{2}=w^{T} \cdot \Sigma \cdot w
$$

where, $\Sigma$ is the covariance matrix

$$
\Sigma=\left(\begin{array}{ccc}
\Sigma_{1,1} & \ldots & \Sigma_{1, n} \\
\vdots & \ddots & \vdots \\
\Sigma_{n, 1} & \ldots & \Sigma_{n, n}
\end{array}\right)
$$

### Portfolio Simulation
Now, we will implement a Monte Carlo simulation to generate random portfolio weights on a larger scale and calculate the expected portfolio return, variance and sharpe ratio for every simulated allocation. We will then identify the portfolio with a highest return for per unit of risk.

In [7]:
def portfolio_Simulation(returns):

    # Calculate the mean returns and covariance matrix
    mean_returns = returns.mean()
    cov_matrix = returns.cov()

    # Simulate 10,000 portfolios
    weights = np.random.random((num_simulations, len(mean_returns)))
    weights /= np.sum(weights, axis=1)[:, np.newaxis]

    # Calculate the portfolio returns and volatility
    portfolio_returns = np.dot(weights, mean_returns) * 260
    portfolio_volatility = np.sqrt(np.sum((np.dot(weights, cov_matrix * 260) * weights), axis=1))

    # Create a DataFrame for analysis
    df = pd.DataFrame({
        'Portfolio Return': portfolio_returns,
        'Portfolio Volatility': portfolio_volatility
    })

    for i, symbol in enumerate(returns.columns):
        df[symbol + ' Weight'] = weights[:, i]

    df['Sharpe Ratio'] = df['Portfolio Return'] / df['Portfolio Volatility']

    # Round the values to 3 decimal places
    df = df.round(4)

    return df

### Maximum Sharpe Ratio Portfolio

In [8]:
portfolio_simulation_df = portfolio_Simulation(returns)
portfolio_simulation_df.head()

,Portfolio Return,Portfolio Volatility,AAPL Weight,AMZN Weight,GOOG Weight,MSFT Weight,Sharpe Ratio
0,0.1308,0.2796,0.2276,0.2966,0.2499,0.2259,0.4678
1,0.1563,0.2796,0.1766,0.2692,0.1824,0.3717,0.5591
2,0.0744,0.2681,0.3612,0.1437,0.2968,0.1983,0.2774
3,0.1623,0.3050,0.3439,0.5604,0.0430,0.0527,0.5322
4,0.1789,0.2899,0.0081,0.3329,0.3111,0.3479,0.6171


In [9]:
# Check results dataframe statistics
portfolio_simulation_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Portfolio Return,10000.0,0.121956,0.045781,-0.0228,0.092500,0.12220,0.151600,0.2697
Portfolio Volatility,10000.0,0.279541,0.011133,0.2618,0.271200,0.27800,0.285600,0.3483
AAPL Weight,10000.0,0.250145,0.142190,0.0000,0.141575,0.24750,0.342200,0.8983
AMZN Weight,10000.0,0.250962,0.140325,0.0002,0.144375,0.25190,0.343000,0.8832
GOOG Weight,10000.0,0.247463,0.141028,0.0002,0.137975,0.24720,0.339800,0.8025
MSFT Weight,10000.0,0.251429,0.139765,0.0000,0.142375,0.25005,0.345225,0.9035
Sharpe Ratio,10000.0,0.432048,0.150120,-0.0821,0.338575,0.44010,0.534000,0.8223


In [10]:
# Get the max sharpe portfolio stats
max_sharpe_portfolio = portfolio_simulation_df.iloc[portfolio_simulation_df['Sharpe Ratio'].idxmax()]
print(max_sharpe_portfolio)

Portfolio Return        0.2455
Portfolio Volatility    0.2986
AAPL Weight             0.0042
AMZN Weight             0.3652
GOOG Weight             0.0075
MSFT Weight             0.6231
Sharpe Ratio            0.8223
Name: 7038, dtype: float64


### Visualise Monte Carlo Portfolio Simulation

In [11]:
# Plot the Monte Carlo Simulation results
fig = px.scatter(
    portfolio_simulation_df, x='Portfolio Volatility', y='Portfolio Return', color='Sharpe Ratio',
    labels={'Portfolio Volatility': 'Volatility', 'Portfolio Return': 'Return', 'Sharpe Ratio': 'Sharpe Ratio'},
    color_continuous_scale=px.colors.sequential.Teal,
    title='Monte Carlo Simulated Portfolios'
    ).update_traces(mode='markers', marker=dict(symbol='circle', size=5))

# Plot the max sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[max_sharpe_portfolio['Portfolio Volatility']],
    y=[max_sharpe_portfolio['Portfolio Return']],
    marker=dict(color='red', size=6, symbol='circle', line=dict(color='red', width=1)),
    name='Max Sharpe Ratio'
).update_layout(showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

### Efficient Frontier
The Efficient Frontier is a set of optimal portfolios in the risk-return spectrum, and portfolios located under the Efficient Frontier curve are considered sub-optimal.

Return objective:

$$
\operatorname{minimize}_{w_{1}, w_{2}, \ldots, w_{n}}^{2}\left(w_{1}, w_{2}, \ldots, w_{n}\right)
$$

subject to,

$$
E\left[R_{p}\right]=m
$$

Risk constraint:

$$
\underset{w_{1}, w_{2}, \ldots, w_{n}}{\operatorname{maximize}} E\left[R_{p}\left(w_{1}, w_{2}, \ldots, w_{n}\right)\right]
$$

subject to,

$$
\sigma_{p}^{2}\left(w_{1}, w_{2}, \ldots, w_{n}\right)=v^{2}
$$

where, $\sum_{i=1}^{n} w_{i}=1$ for the above objectives.

We can use numerical optimisation to achieve this objective. The goal is to find the optimal value of the objective function by adjusting the target variables operating within some boundary conditions and constraints.

### Constrained Optimization
Construction of optimal portfolios is a constrained optimisation problem where we specify some boundary conditions and constraints. The objective function here is a function returning maximum sharpe ratio, minimum variance (volatility) and the target variables are portfolio weights. We will use the minimize function from scipy optimization module to achieve our objective.

In [12]:
import scipy.optimize as sco

### Efficient Frontier Portfolio
For efficient portfolios we fix a target retrun and derive for the object function

In [13]:
# Specify the constraints, bounds and initial guess of the weights
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for x in range(num_assets))
inital_weights = np.array([1/num_assets]*num_assets)

In [14]:
# Optimise for the Max Sharpe Ratio portfolio
max_sharpe_opt = sco.minimize(fun=portfolio_Negative_Max_Sharpe_Ratio, x0=inital_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights
max_sharpe_weights = max_sharpe_opt['x'].round(4)

# Get the portfolio performance
max_sharpe_performance_df = portfolio_Performance(max_sharpe_weights)

# Print stats
print(f'Max Sharpe Ratio Performance: ')
print(max_sharpe_performance_df, '\n')
print(f'Max Sharpe Weights: {max_sharpe_weights}')

Max Sharpe Ratio Performance: 
     Return  Volatility  Sharpe Ratio
0  0.251703    0.306148      0.822163 

Max Sharpe Weights: [0.     0.5225 0.     0.4775]


In [15]:
# Optimise for the minimum variance portfolio
min_variance_opt = sco.minimize(fun=portfolio_Variance, x0=inital_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights
min_variance_weights = min_variance_opt['x'].round(4)

# Get the portfolio performance
min_variance_performance_df = portfolio_Performance(min_variance_weights)

# Print stats
print(f'Min Variance Performance: ')
print(min_variance_performance_df, '\n')
print(f'Min Variance Weights: {min_variance_weights}')

Min Variance Performance: 
     Return  Volatility  Sharpe Ratio
0  0.047234    0.257718      0.183276 

Min Variance Weights: [0.3911 0.     0.3313 0.2776]


In [16]:
# Define a function to optimise for the minimum variance portfolio given a target return
def optimise_Minimum_Volatility(target_return: float, bounds: tuple, initial_weights: np.ndarray):
    ''' Optimises to find the weights of a minimum variance portfolio for a given target return '''
    # Define the constaints, bounds and initial guess of the weights
    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_Return(x) - target_return}, 
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

    return sco.minimize(fun=portfolio_Volatility, x0=initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)['fun']

# Create a list of returns to optimise for
target_returns = np.linspace(min_variance_performance_df['Return'][0], max_sharpe_performance_df['Return'][0], 100)

# All portfolios have the same constraints, bounds and initial weights guess
bounds = tuple((0, 1) for x in range(num_assets))
inital_weights = np.array([1/num_assets]*num_assets)

# Create a list of minimum volatilities for each target return
minimum_volatilities = np.array([optimise_Minimum_Volatility(target_return, bounds, inital_weights) for target_return in target_returns])

sharpe_ratios = target_returns/minimum_volatilities

# Create a dataframe of the target returns and minimum volatilities
efficient_frontier_df = pd.DataFrame({
    'Target Returns': target_returns,
    'Target Volatility': minimum_volatilities,
    'Target Sharpe Ratio': target_returns/minimum_volatilities
})

efficient_frontier_df.head(10)

,Target Returns,Target Volatility,Target Sharpe Ratio
0,0.047234,0.257715,0.183278
1,0.049299,0.257721,0.191287
2,0.051364,0.257736,0.199290
3,0.053430,0.257758,0.207286
4,0.055495,0.257788,0.215273
5,0.057560,0.257826,0.223252
6,0.059626,0.257873,0.231221
7,0.061691,0.257927,0.239180
8,0.063756,0.257989,0.247128
9,0.065822,0.258059,0.255065


In [17]:
# Plot the efficient frontier portfolio
fig = px.scatter(
    efficient_frontier_df, x='Target Volatility', y='Target Returns', color='Target Sharpe Ratio',
    labels={'Target Volatility': 'Volatility', 'Target Returns': 'Return', 'Target Sharpe Ratio': 'Sharpe Ratio'},
    color_continuous_scale=px.colors.sequential.Teal,
    title='Efficient Frontier Mean-Variance Optimised Portfolios'
    ).update_traces(mode='markers', marker=dict(symbol='circle', size=5))

# Plot the max sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=max_sharpe_performance_df['Volatility'],
    y=max_sharpe_performance_df['Return'],
    marker=dict(color='red', size=6, symbol='circle', line=dict(color='red', width=1)),
    name='Max Sharpe Ratio'
).update_layout(showlegend=False)

# Plot the min variance portfolio
fig.add_scatter(
    mode='markers',
    x=min_variance_performance_df['Volatility'],
    y=min_variance_performance_df['Return'],
    marker=dict(color='green', size=6, symbol='circle', line=dict(color='green', width=1)),
    name='Minimum Variance'
).update_layout(showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()